<a href="https://colab.research.google.com/github/sebasDev2001/ProyectoFinal/blob/main/Proyecto_Fox_and_Hounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Fox and Hounds**

<img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/4/42/Fox_and_hounds.svg/220px-Fox_and_hounds.svg.png">

The game consists of hounds who have to prevent a fox form moving to ther other side of the board. The fox starts on the bottom left of the board and the hounds start on the evens squares at the top of the board.

The hounds move diagonaly foward and the fox moves diagonaly foward or backward.

In [ ]:
import pydot
from IPython.display import Image, display
import queue
import numpy as np

In [ ]:
class Node ():
  def __init__(self, state,value,operators,operator=None, parent=None,objective=None):
    self.state= state
    self.value = value
    self.children = []
    self.parent=parent
    self.operator=operator
    self.objective=objective
    self.level=0
    self.operators=operators
    self.v=0

    
  def add_child(self, value, state, operator):
    node=type(self)(value=value, state=state, operator=operator,parent=self,operators=self.operators)
    node.level=node.parent.level+1
    self.children.append(node)
    return node
  
  def add_node_child(self, node):
    node.level=node.parent.level+1
    self.children.append(node)    
    return node

  #Devuelve todos los estados según los operadores aplicados
  def getchildrens(self):
    print(self.operators)
    return [
        self.getState(i) 
           for i, op in enumerate(self.operators)]
    
  def getState(self, index):
    pass
  
  def __eq__(self, other):
    return self.state == other.state
 
  def __lt__(self, other):
    return self.f() < other.f()
   
  
  def repeatStatePath(self, state):
      n=self
      while n is not None and n.state!=state:
          n=n.parent
      return n is not None
    
  def pathObjective(self):
      n=self
      result=[]
      while n is not None:
          result.append(n)
          n=n.parent
      return result
  
  def heuristic(self):
    return 0
  
  def cost(self):
    return 1
  
  def f(self): 
    return self.cost()+self.heuristic()

  ### Crear método para criterio objetivo
  ### Por defecto vamos a poner que sea igual al estado objetivo, para cada caso se puede sobreescribir la función
  def isObjective(self):
    return 0
    #return (self.state==self.objetive.state)

In [ ]:
class Tree ():
  def __init__(self, root ,operators):
    self.root=root
    self.operators=operators

  def printPath(self,n):
    stack=n.pathObjective()
    path=stack.copy()
    while len(stack)!=0:
        node=stack.pop()
        if node.operator is not None:
            print(f'operador:  {self.operators[node.operator]} \t estado: {node.state}')
        else:
            print(f' {node.state}')
    return path

  def reinitRoot(self):
    self.root.operator=None
    self.root.parent=None
    self.root.objective=None
    self.root.children = []
    self.root.level=0
    
  def miniMax(self, depth):
    self.root.v=self.miniMaxR(self.root, depth, True)
    ## Comparar los hijos de root
    values=[c.v for c in self.root.children]
    maxvalue=max(values)
    index=values.index(maxvalue)
    return self.root.children[index]

  def miniMaxR(self, node, depth, maxPlayer):
    if depth==0 or node.isObjective():
      node.v=node.heuristic()
      return node.heuristic()
    ## Generar los hijos del nodo
    children=node.getchildrens()
    
    ## Según el jugador que sea en el árbol
    if maxPlayer:
      value=float('-inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)
          value=max(value,self.miniMaxR(newChild,depth-1,False))
      #node.v=value
      #return value
    else:
      value=float('inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)
          value=min(value,self.miniMaxR(newChild,depth-1,True))
    node.v=value
    return value

  def alphaBeta(self, depth):
    self.root.v= self.alphaBetaR(self.root, depth,float('-inf'),float('+inf'), True)
    values =[c.v for c in self.root.children]
    maxvalue=max(values)
    index = values.index(maxvalue)
    return self.root.children[index]

  def alphaBetaR(self, node, depth,alpha,beta, maxPlayer):
    if depth==0 or node.isObjective():
      node.v = node.heuristic()
      return node.heuristic()

    children = node.getchildrens()
    #print("Hijos:",children)
    if maxPlayer:
      value=float('-inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=False)
          newChild=node.add_node_child(newChild)   
          value = max(value,self.alphaBetaR(newChild, depth-1, alpha,beta,False))
          alpha = max(alpha,value)
          if alpha>=beta:
            break
    else:
      value=float('inf')
      for i,child in enumerate(children):
        if child is not None:
          newChild=type(self.root)(value=node.value+'-'+str(i),state=child,operator=i,parent=node, operators=node.operators,player=True)
          newChild=node.add_node_child(newChild)   
          value = min(value,self.alphaBetaR(newChild, depth-1, alpha,beta,True))
          beta = min(beta,value)
          if alpha>=beta:
            break
    node.v=value
    return value

  ## Método para dibujar el árbol        
  def draw(self,path):
    graph = pydot.Dot(graph_type='graph')
    nodeGraph=pydot.Node(str(self.root.state)+"-"+str(0),
                          label=str(self.root.state),shape ="circle", 
                          style="filled", fillcolor="red")
    graph.add_node(nodeGraph)
    path.pop()
    return self.drawTreeRec(self.root,nodeGraph,graph,0,path.pop(),path)
  
  ## Método recursivo para dibujar el árbol  
  def drawTreeRec(self,r,rootGraph,graph,i,topPath,path):
    if r is not None:
      children=r.children
      for j,child in enumerate(children):
        i=i+1
        color="white"
        if topPath.value==child.value:
          if len(path)>0:topPath=path.pop()
          color='red'
        c=pydot.Node(child.value,label=str(child.state)+r"\n"+r"\n"+"f="+str(child.heuristic())+r"\n"+str(child.v), 
                      shape ="circle", style="filled", 
                      fillcolor=color)
        graph.add_node(c)
        graph.add_edge(pydot.Edge(rootGraph, c, 
                                  label=str(child.operator)+'('+str(child.cost())+')'))
        graph=self.drawTreeRec(child,c,graph,i,topPath,path)  # recursive call
      return graph
    else:
      return graph  

In [ ]:
class NodeFoxandHounds(Node):
  ## Vamos a añadir el jugador, pues en dependencia del jugador se hace una cosa u otra.

  def __init__(self, player=True,**kwargs):
    super(NodeFoxandHounds, self).__init__(**kwargs)
    self.player=player
    if player:
      self.v=float('-inf')
    else:
      self.v=float('inf')
  
  def heuristica(self,state):
    (x,y) = buscarZorro(state)
    distanciasAPerros = 0
    for i in range(len(initState)):
      for j in range(len(initState[i])):
        if state[i][j] != 0 and state[i][j] != 'Z':
          n = (i-x)**2 + (j-y)**2
          distancia = np.sqrt(n)
          distanciasAPerros += distancia


    distanciasAPerros = distanciasAPerros/4
    distanciaObjetivo = x+1
    distanciaIzquiera = y+1
    distanciaDerecha = abs(len(initState[x])-(y+1))   

    return (distanciasAPerros*distanciaDerecha*distanciaIzquiera)/distanciaObjetivo 

  def buscarZorro(self,state):
    for i in range(len(initState)):
      for j in range(len(initState[0])):
        if state[i][j] == 'Z':
          return (i,j)
  
  

  def getState(self,index): 
    estado = self.state
    (x,y) = self.operators[index]
    
    jugadasP = self.validoZ(estado)
    if self.operators[index] in jugadasP:
      print('inside')
      print(x,y)
      nEstado= self.NuevoEstadoZ(estado,x,y)
      print('print',nEstado)
      return nEstado
    return None


  def validoZ (self,estado): 
    fila = 0
    jugadas = []
    while fila < len(estado):
      columna = 0
      while columna < len(estado[fila]):
        if estado[fila][columna]== "Z":
          if (fila != 0): ## primera fila
            if (columna != 0): ## Columna izquierda
              if estado [fila-1][columna-1] ==0 :
                jugadas.append((fila-1,columna-1))
            if (columna != 7): ## Columna derecha
              if estado [fila-1][columna+1] ==0 :
                jugadas.append((fila-1,columna+1))
          if (fila != 7): ## ultima fila
            if (columna != 0): ## Columna izquierda
              if estado [fila+1][columna-1] ==0 :
                jugadas.append((fila+1,columna-1))
            if (columna != 7): ## Columna derecha
              if estado [fila+1][columna+1] ==0 :
                jugadas.append((fila+1,columna+1))
        columna = columna +1
      fila = fila +1
    return jugadas


  def NuevoEstadoZ(self,estado1,fila1,columna1): ## Ya esta verificado que si se pueda hacer el movimiento
    estado = estado1.copy()
    fila = 0
    while fila < len(estado):
      columna = 0
      while columna < len(estado[fila]):
        if estado[fila][columna]== 'Z':
          estado[fila][columna] = 0
        if fila1 == fila and columna1==columna :
          estado[fila][columna] = 'Z'
        columna = columna +1
      fila = fila +1
    print(estado)
    return estado

  def operadoresZ (self,estado): # devuelve todas las casillas negras
    fila = 0
    operadores = []
    while fila < len(estado):
      columna = 0
      while columna < len(estado[fila]):
        if fila %2 == 0 : 
          if (columna%2 != 0): 
            operadores.append((fila,columna))
        else: 
          if (columna%2 == 0): 
            operadores.append((fila,columna))
        columna = columna +1
      fila = fila +1
    return operadores
  
  def cantMove(self,state):
    (x,y) = self.buscarZorro(state)
    if x==len(state)-1 and y==0:
      if state[x-1][y+1] !=0:
        return True
    elif x==len(state)-1 and y==len(state[x])-1:
      if state[x-1][y-1] !=0:
        return True
    elif y==0:
      if state[x-1][y+1] !=0 and state[x+1][y+1] !=0:
        return True
    elif y==len(state[x])-1:
      if state[x-1][y-1] !=0 and state[x+1][y-1] !=0:
        return True
    elif x==len(state)-1:
      if state[x-1][y-1] !=0 and state[x-1][y+1] !=0:
        return True
    elif state[x-1][y-1] !=0 and state[x-1][y+1] !=0 and state[x+1][y-1] !=0 and state[x+1][y+1] !=0:
      return True
    return False
  
  def isObjective(self):
    (x,y) = self.buscarZorro(self.state)
    if x == 0:
      return True;
    elif self.cantMove(self.state):
      return True
    return False

In [ ]:
class Juego:

  def printState(self,state):
    for i in state:
      aux = '|'
      for j in i:
        aux = aux +str(j)+'|'
      print(aux)
      #print('-----------------')
      

  def menu(self,initState):
    print("Elija la dificultad del juego: ")
    print("1.Facil")
    print("2.Moderado")
    print("3.Dificil")
    print("4.Terminar")
    x = input()
    dif = 0
    if int(x) == 1:
      dif = 1
    elif int(x) == 2:
      dif = 4
    elif int(x) == 3:
      dif = -1
    if dif == 0:
      return 0
    return self.turnos(dif,initState)
  
  def operadoresZ (self,estado): # devuelve todas las casillas negras
    fila = 0
    operadores = []
    while fila < len(estado):
      columna = 0
      while columna < len(estado[fila]):
        if fila %2 == 0 : 
          if (columna%2 != 0): 
            operadores.append((fila,columna))
        else: 
          if (columna%2 == 0): 
            operadores.append((fila,columna))
        columna = columna +1
      fila = fila +1
    return operadores
  
  def turnos(self,dif,initState):
    operators = self.operadoresZ(initState)
    state = NodeFoxandHounds(True,value = "Inicio",state = initState,operators=operators)
    self.printState(state.state)
    while not state.isObjective():
      #Introducir posicion
      temp = True
      aux = [1,2,3,4]
      while temp:
        print("Introducir que Hound se va a mover: 1-4")
        x = input()
        print("Introducir la direccion a la que desea moverse: ")
        print("1- Izquierda Inferior")
        print("2- Derecha inferior")
        y = input()
        try:
          x = int(x)
          y = int(y)
          #Cambiar el estado
          if x < 8 and y < 8:
            for i in range(len(state.state)):
              try:
                index = state.state[i].index(x)
                if index != None:
                  break
              except Exception:
                pass
            for j in aux:
              if x == j and y == 1:
                if index != 0:
                  if state.state[i+1][index-1] == 0:
                    state.state[i][index] = 0
                    state.state[i+1][index-1] = x
                    temp = False
                  else:
                    print("Ya casilla ya esta ocupada")
                else:
                  print("El movimiento no es posible")
              elif x == j and y == 2:
                if index != 7:
                  if state.state[i+1][index+1] == 0:
                    state.state[i][index] = 0
                    state.state[i+1][index+1] = x
                    temp = False
                  else:
                    print("Ya casilla ya esta ocupada")
                else:
                  print("El movimiento no es posible")
            else:
              print("La posicion ingresada no fue valida")
        except Exception:
          print("El dato ingresado no es valido")
      self.printState(state.state)
      #Juega IA(Ejecutar Alfabeta) y actualiza el estado
      #self.printState(state.state)
      treeMinMax= Tree(state,operators)
      #print(treeMinMax.root.state)
      state = treeMinMax.alphaBeta(dif)
      #Imprimir estado
      self.printState(state.state)
      #print(state.value)
      state = NodeFoxandHound(True,value = state.value,state = state.state,operators=operators)
  #return state.state

In [ ]:
initState = [[ 0 , 1 , 0 , 2 , 0 , 3 , 0 , 4 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 ,'Z', 0 , 0 , 0 , 0 , 0 , 0 ],
             [ 0 , 0 , 0 , 0 , 0 , 0 , 0 , 0 ]]
juego = Juego()
#juego.menu(initState)
operators = juego.operadoresZ(initState)
n = NodeFoxandHounds(True,value = "Inicio",state = initState,operators=operators)
n.getchildrens()

[(0, 1), (0, 3), (0, 5), (0, 7), (1, 0), (1, 2), (1, 4), (1, 6), (2, 1), (2, 3), (2, 5), (2, 7), (3, 0), (3, 2), (3, 4), (3, 6), (4, 1), (4, 3), (4, 5), (4, 7), (5, 0), (5, 2), (5, 4), (5, 6), (6, 1), (6, 3), (6, 5), (6, 7), (7, 0), (7, 2), (7, 4), (7, 6)]
inside
5 0
[[0, 1, 0, 2, 0, 3, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], ['Z', 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
print [[0, 1, 0, 2, 0, 3, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], ['Z', 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
inside
6 1
[[0, 1, 0, 2, 0, 3, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 'Z', 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]
print [[0, 1, 0, 2, 0, 3, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 [[0, 1, 0, 2, 0, 3, 0, 4],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  ['Z', 0, 0, 0, 0, 0, 0, 0]],
 None,
 None,
 None,
 [[0, 1, 0, 2, 0, 3, 0, 4],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  ['Z', 0, 0, 0, 0, 0, 0, 0]],
 None,
 None,
 None,
 [[0, 1, 0, 2, 0, 3, 0, 4],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  [0, 0, 0, 0, 0, 0, 0, 0],
  ['Z', 0, 0, 0, 0, 0, 0, 0]],
 None,
 None,
 None]

In [ ]:
[[0, 1, 0, 2, 0, 3, 0, 4], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], ['Z', 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0]]

[[0, 1, 0, 2, 0, 3, 0, 4],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 ['Z', 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0]]